## Install and import required packages

In [ ]:
%pip install qiskit qiskit-algorithms numpy pandas seaborn

from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit.quantum_info import SparsePauliOp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

## Calculate reference value

In [ ]:
c0 = -0.80718
c1 = 0.17374
c2 = -0.23047
c3 = 0.12149
c4 = 0.16940
c5 = -0.04509
c6 = 0.04509
c7 = 0.16658
c8 = 0.17511

hamiltonian = SparsePauliOp.from_list([
    ("IIII", c0),
    ("ZIII", c1),
    ("ZZII", c2),
    ("IIZI", c1),
    ("IZZZ", c2),
    ("IZII", c3),
    ("ZIZI", c4),
    ("XZXI", c5),
    ("XIXZ", c6),
    ("XIXI", c6),
    ("XZXZ", c5),
    ("ZZZZ", c7),
    ("ZZZI", c7),
    ("ZIZZ", c8),
    ("IZIZ", c3),
])

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=hamiltonian)
ref_value = result.eigenvalue.real
ref_value

## Define helper functions

In [ ]:
def calculate_precision_probability(group, reference_value, precision):
  in_precision_range = abs(group['optimal_value'] - reference_value) <= precision
  num_in_precision = in_precision_range.sum()
  total_count = len(group)
  return num_in_precision / total_count if total_count > 0 else 0

def reached_chemical_precision(row, reference_value, precision):
  return abs(row['optimal_value'] - reference_value) <= precision


def calculate_difference_from_ref_value(group, reference_value):
  diff = abs(group['optimal_value'] - reference_value)
  return diff.mean()

def calculate_avg_function_evals(group):
  in_precision_range = abs(group['optimal_value'] - reference_value) <= precision
  num_in_precision = in_precision_range.sum()
  total_count = len(group)
  return num_in_precision / total_count if total_count > 0 else 0

## Read produced dataset

In [ ]:
df = pd.read_csv('data.csv')
# add column that tells whether the chemical precision was reached
df['reached'] = df.apply(reached_chemical_precision, reference_value=ref_value, precision=0.0016, axis=1)

# Visualizations

### Impact of ansatz layers

In [ ]:
gr2 = df.groupby(['optimizer', 'num_layers', 'is_gradient_ignored'], sort=False)
df2 = gr2.apply(calculate_precision_probability, reference_value=ref_value, precision=0.0016).reset_index(name='prob_chem_precision')

sns.reset_defaults()
sns.set_theme(rc={'figure.figsize':(9, 6)})
sns.set_style(style='whitegrid')
sns.set_palette(['darkorange', 'gold', 'fuchsia', 'dodgerblue', 'teal', 'springgreen'])

g = sns.scatterplot(x="optimizer",
                    palette=['fuchsia', 'dodgerblue', 'springgreen'],
                    y="prob_chem_precision",
                    hue="num_layers",
                    hue_order=[1,2,3],
                    style='is_gradient_ignored',
                    data=df2,
                    s=100)

g.set_xlabel("Optimizers",fontsize=13)
g.set_ylabel("Pr[chemical precision]", fontsize=13)
g.tick_params(labelsize=13)

plt.title('Impact of ansatz layers', y=1.155, fontsize=13)
g.set_xticklabels(g.get_xticklabels(),rotation=30,ha="right",rotation_mode='anchor')
g.set_yticks([x/10 for x in range(11)])
handles, labels = g.get_legend_handles_labels()
del handles[0]
del handles[3]
g.legend(handles, ['1 layer', '2 layers', '3 layers', 'gradient-based', 'gradient-free'], fontsize=13, ncol=5, loc='center', bbox_to_anchor=(0.5, 1.09), columnspacing=0.7)

plt.tight_layout()
plt.savefig('layers.pdf',bbox_inches='tight', dpi=2000)

### Performance of various ansatzes


In [ ]:
df1 = df.loc[df['num_layers'] != 1]
gr1 = df1.groupby(['optimizer', 'entanglement', 'num_layers'], sort=False)
df2 = gr1.apply(calculate_precision_probability, reference_value=ref_value, precision=0.0016).reset_index(name='prob_chem_precision')

sns.reset_defaults()
sns.set_style("whitegrid")
sns.set_palette(['orangered', 'gold', 'fuchsia', 'dodgerblue', 'teal', 'springgreen'])
plt.rcParams['xtick.labelsize'] = 13

f, ax = plt.subplots(figsize=(9, 8))

g = sns.scatterplot(x="optimizer",
                    y="prob_chem_precision",
                    style='num_layers',
                    hue="entanglement",
                    data=df2,
                    ax=ax,
                    s=100)

g.set_xlabel("",fontsize=13, y=-2)
g.set_ylabel("Pr[chemical precision]",fontsize=13)
g.tick_params(labelsize=13)

plt.title('Performance of various ansatzes', y=1.28, fontsize=13)
ax.set_xticklabels(ax.get_xticklabels(),rotation=30, ha="right", rotation_mode='anchor')
handles, labels = ax.get_legend_handles_labels()
del handles[0]
del handles[6]

labels=['linear', 'reverse linear', 'full', 'circular', 'pairwise', 'sca', '2 layers', '3 layers']
ax.legend(fontsize=13,handles=handles, labels=labels, ncol=4, loc='center', bbox_to_anchor=(0.5, 1.15), columnspacing=0.8)
ax.set_yticks([x/10 for x in range(11)]);

ax2 = ax.twiny()
ax2.spines["bottom"].set_position(("axes", -0.30))
ax2.tick_params('both', length=0, width=0, which='minor')
ax2.tick_params('both', direction='in', which='major')
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

ax2.set_xticks([0.0, 0.53, 1.0])
ax2.xaxis.set_major_formatter(ticker.NullFormatter())
ax2.xaxis.set_minor_locator(ticker.FixedLocator([0.28, 0.77]))
ax2.xaxis.set_minor_formatter(ticker.FixedFormatter(['Gradient-free optimizers', 'Gradient-based optimizers']))
ax2.tick_params(which='major', length=7)

plt.tight_layout()
plt.savefig('chemical.pdf', dpi=2000)

### Example of energy convergence

In [ ]:
df_cobyla = df.loc[(df['optimizer'] == 'COBYLA') & (df['num_layers'] == 2) & (df['sample_id'] == 0)]
df_gd = df.loc[(df['optimizer'] == 'GradientDescent') & (df['num_layers'] == 2) & (df['sample_id'] == 0)]

data = []
for index, row in df_cobyla.iterrows():
  convergence = list(eval(row['energy_convergence']))
  fun_evals, energy = zip(*convergence)
  for i in range(len(fun_evals)):
    data.append((energy[i],fun_evals[i],row['optimizer'],row['entanglement']))

df_cobyla = pd.DataFrame(data, columns=['energy', 'fun_evals', 'optimizer', 'entanglement'])

data = []
for index, row in df_gd.iterrows():
  convergence = list(eval(row['energy_convergence']))
  fun_evals, energy = zip(*convergence)
  for i in range(len(fun_evals)):
    data.append((energy[i],fun_evals[i],row['optimizer'],row['entanglement']))

df_gd = pd.DataFrame(data, columns=['energy', 'fun_evals', 'optimizer', 'entanglement'])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 4), sharey=True)

sns.reset_defaults()
sns.set_style(style='white')
sns.set_palette(['darkorange', 'gold', 'fuchsia', 'dodgerblue', 'teal', 'springgreen'])

sns.lineplot(df_cobyla, ax=axes[0], x="fun_evals", y="energy", hue="entanglement")
axes[0].set_title('COBYLA', fontsize=13)
axes[0].set_xlabel('')
axes[0].set_ylabel('')
axes[0].axhline(y = ref_value, color = 'red', linestyle = '--', label='reference value')
axes[0].xaxis.set_tick_params(labelsize=13)
axes[0].tick_params(axis='y', labelsize=13)
axes[0].xaxis.set_major_locator(ticker.MultipleLocator(25))
axes[0].yaxis.set_major_locator(ticker.MultipleLocator(0.2))
axes[0].get_legend().remove()

sns.lineplot(df_gd, ax=axes[1], x="fun_evals", y="energy", hue="entanglement")
axes[1].set_title('Gradient Descent', fontsize=13)
axes[1].set_xlabel('')
axes[1].axhline(y = ref_value, color = 'red', linestyle = '--', label='reference value')
axes[1].xaxis.set_tick_params(labelsize=13)
axes[1].yaxis.set_tick_params(labelsize=13)
axes[1].get_legend().remove()
axes[1].xaxis.set_major_locator(ticker.MultipleLocator(250))
fig.supxlabel('Cost function evaluations', y=-0.0015, fontsize=13)
fig.supylabel('Energy [Ha]', fontsize=13)
fig.suptitle('Energy convergence for two optimizers', y=1.16, fontsize=13)

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, fontsize=13, ncol=4, loc='center', bbox_to_anchor=(0.5, 1), columnspacing=0.8)
fig.tight_layout()
plt.savefig('convergence_example.pdf', bbox_inches='tight', dpi=2000)

### An average number of times the chemical precision is reached in an hour

In [ ]:
df1 = df.loc[df['num_layers'] != 1]
df2 = df1.copy().groupby(['optimizer', 'is_gradient_ignored'], sort=False, as_index=False).agg({'optimizer_time': 'sum', 'reached': 'sum'})
df2['hours'] = df2.apply(lambda row: 60/(row.optimizer_time/row.reached) if row.reached != 0 else 0, axis=1)

sns.reset_defaults()
sns.set_style("whitegrid")
sns.set_palette(['dodgerblue','springgreen'])

f, ax = plt.subplots(figsize=(8, 6))

g = sns.scatterplot(x="optimizer",
                    y="hours",
                    hue="is_gradient_ignored",
                    data=df2,
                    s=100)

plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
g.set_xlabel("Optimizers", fontsize=13)
ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha="right", rotation_mode='anchor')
handles, labels = ax.get_legend_handles_labels()
labels=['gradient-free', 'gradient-based']
ax.legend(fontsize=13, handles=handles, labels=labels, ncol=2, loc='center', bbox_to_anchor=(0.5, 1.07), columnspacing=0.8)

plt.title('An average number of times the chemical precision is reached in an hour', y=1.13, fontsize=13)
ax.set_ylabel("", fontsize=13)
plt.yticks(np.arange(0, 22.51, step=2.5), fontsize=13, va='center')

plt.tight_layout()
plt.savefig('hours.pdf', dpi=2000)

### Cost function evaluations with the probability of reaching the chemical precision

In [ ]:
df1 = df.loc[df['num_layers'] != 1]
df2 = df1.groupby(['optimizer'], sort=False)
g1 = df1.groupby(['optimizer', 'is_gradient_ignored'], sort=False)
df3 = g1.apply(calculate_precision_probability, reference_value=ref_value, precision=0.0016).reset_index(name='prob_chem_precision')

sns.reset_defaults()
sns.set_style("white")
sns.set_palette(['dodgerblue', 'springgreen', 'darkorange'])

f, ax = plt.subplots(figsize=(8, 6))

g = sns.boxplot(x="optimizer",
                y="cost_function_evals",
                hue="is_gradient_ignored",
                data=df1,
                ax=ax)

plt.yticks(fontsize=13)
g.set_xlabel("Optimizers",fontsize=13)

ax2 = ax.twinx()
g = sns.scatterplot(x="optimizer",
                    y="prob_chem_precision",
                    data=df3,
                    color='darkorange',
                    s=100,
                    ax=ax2)

ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha="right", rotation_mode='anchor')
handles, labels = ax.get_legend_handles_labels()
handles.append(plt.Line2D([0], [0], marker='o', color='darkorange', markersize=8, linestyle='None'))
labels=['gradient-free', 'gradient-based', 'Pr[chemical precision]']
ax.legend(fontsize=13, handles=handles, labels=labels, ncol=3, loc='center', bbox_to_anchor=(0.5, 1.07), columnspacing=0.8)

plt.title('Cost function evaluations with the probability of reaching the chemical precision', y=1.13, fontsize=13)
ax.set_ylabel("Cost function evaluations", fontsize=13)
ax2.set_ylabel('Pr[chemical precision]', fontsize=13)

ax2.margins(y=0.020)
ax2.tick_params(direction='out', pad=15)

plt.yticks(np.arange(0, 1.1, step=0.1), fontsize=13, va='center')
plt.tight_layout()
plt.savefig('evaluations.pdf', dpi=2000)
plt.show()